In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.pydantic_v1 import BaseModel, Field
wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)


class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )


tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=wrapper,
    return_direct=True,
)

print(tool.run("langchain"))

Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications 


In [4]:
print(f"Name: {tool.name}")
print(f"Description: {tool.description}")
print(f"args schema: {tool.args}")
print(f"returns directly?: {tool.return_direct}")

Name: wiki-tool
Description: look up things in wikipedia
args schema: {'query': {'title': 'Query', 'description': 'query to look up in Wikipedia, should be 3 or less words', 'type': 'string'}}
returns directly?: True


In [6]:
from langchain_core.tools import tool
@tool
def WikipediaAPIWrapperTool(query:str): 
    """Tool to perform wiki search"""
    wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
    tool = WikipediaQueryRun(api_wrapper=wrapper)
    return tool.invoke(query)


In [7]:
from langchain_openai import ChatOpenAI

tools = [WikipediaAPIWrapperTool]

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [8]:
from langchain import hub
from langchain.agents import create_tool_calling_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

agent = create_tool_calling_agent(llm, tools, prompt)

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [9]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
query = "Tell me about langchain and langgraph"


# response = agent_executor.invoke(query)

agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `WikipediaAPIWrapperTool` with `{'query': 'LangChain'}`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications 
Invoking: `WikipediaAPIWrapperTool` with `{'query': 'LangGraph'}`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications **LangChain** is a framework designed to simplify the creation of applications that involve language models. It provides tools and components that help developers build applications that utilize natural language processing capabilities more effectively.

As for **LangGraph**, I couldn't find specific information. It may not have a dedicated Wikipedia page or may not be widely recognized yet. If you have any specific aspects or context about LangGraph that you would like to explore, please let me know!

> Finished chain.


{'input': 'Tell me about langchain and langgraph',
 'output': "**LangChain** is a framework designed to simplify the creation of applications that involve language models. It provides tools and components that help developers build applications that utilize natural language processing capabilities more effectively.\n\nAs for **LangGraph**, I couldn't find specific information. It may not have a dedicated Wikipedia page or may not be widely recognized yet. If you have any specific aspects or context about LangGraph that you would like to explore, please let me know!"}